## Running your own LLM

This notebook was adapted from Sil Hamilton's class [Generative AI for journalists](https://www.kccourses.org/enrol/index.php?id=116). 

We first download the required software: LangChain and its dependency `pypdf`

In [11]:
#!pip install --upgrade pip 
#!pip install --upgrade langchain pypdf
#!pip install -U langchain-community
!pip install -U langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 2.6 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


We then load LangChain's `pypdf` loader.

In [12]:
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

Now let's load PDFs

In [13]:
# loader = PyPDFLoader("../data/2021-census-population-occupied-private-dwellings-community-2001-2021.pdf")

In [14]:
# single_pdf = loader.load_and_split()

In [15]:
loader = PyPDFDirectoryLoader("../data/Supreme Court opinions 2014/")

In [16]:
many_pdfs = loader.load_and_split()

Having loaded our data, we'll now download and load the embedding model.

In [17]:
#!pip install sentence_transformers > /dev/null

In [19]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [21]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Let's try embedding some text. Observe the output. Once you've tried it, scroll down to continue.

In [22]:
text = "This is a test document."

In [23]:
embeddings.embed_query(text)

[-0.03833850845694542,
 0.1234646588563919,
 -0.02864295430481434,
 0.05365273728966713,
 0.008845346979796886,
 -0.03983934596180916,
 -0.07300586998462677,
 0.04777126759290695,
 -0.03046250157058239,
 0.05497973784804344,
 0.08505290001630783,
 0.03665672987699509,
 -0.005319980438798666,
 -0.0022331627551466227,
 -0.06071098521351814,
 -0.027237940579652786,
 -0.011351647786796093,
 -0.04243769869208336,
 0.009129912592470646,
 0.10081558674573898,
 0.07578732818365097,
 0.06911719590425491,
 0.009857515804469585,
 -0.0018377389060333371,
 0.026249056681990623,
 0.03290240094065666,
 -0.07177440077066422,
 0.02838428132236004,
 0.061709512025117874,
 -0.052529554814100266,
 0.03366169333457947,
 0.07446811348199844,
 0.07536035776138306,
 0.03538399934768677,
 0.06713403761386871,
 0.010798030532896519,
 0.08167024701833725,
 0.01656290702521801,
 0.032830629497766495,
 0.03632565960288048,
 0.002172861248254776,
 -0.09895740449428558,
 0.005046738777309656,
 0.05089650675654411,
 

We now have a working embedding function. Let's install Chroma.

In [24]:
#!pip install -U chromadb

In [25]:
from langchain.vectorstores import Chroma

Let's make a vector store for our loaded documents!

In [27]:
%%time
db = Chroma.from_documents(many_pdfs, embeddings)

CPU times: user 60 s, sys: 1.87 s, total: 1min 1s
Wall time: 50 s


Let's try retrieving a relevant document.

In [28]:
query = "What documents include Sotomayor?"
db.similarity_search(query)

[Document(metadata={'page': 11, 'source': '../data/Supreme Court opinions 2014/13-433_5h26.pdf'}, page_content='_________________ \n \n_________________ \n \n \n \n \n \n \n  \n \n  \n  \n \n \n \n \n \n \n \n1 Cite as: 574 U. S. ____ (2014) \nSOTOMAYOR, J., concurring \nSUPREME COURT OF THE UNITED STATES \nNo. 13–433 \nINTEGRITY STAFFING SOLUTIONS, INC., \nPETITIONER v. JESSE BUSK ET AL. \nON WRIT OF CERTIORARI TO THE UNITED STATES COURT OF \nAPPEALS FOR THE NINTH CIRCUIT\n \n[December 9, 2014]\n JUSTICE SOTOMAYOR, with whom J USTICE KAGAN joins,\nconcurring. \nI concur in the Court’s opinion, and write separately\nonly to explain my understanding of the standards the\nCourt applies. \nThe Court reaches two critical conclusions.  First, the \nCourt confirms that compensable “ ‘principal’” activities \n“‘includ[e] . . . those closely related activities which are \nindispensable to [a principal activity’s] performance,’ ” \nante, at 6 (quoting 29 CFR §790.8(c)(2013)), and holds that \nt